In [1]:
import pandas as pd

C:\anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
import numpy as np
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import pandas as pd

file_path = "C:\\Users\\user\\Downloads\\covid\\Netflix Dataset.csv"

data = pd.read_csv(file_path)
print(data.head())

  Show_Id Category  Title           Director  \
0      s1  TV Show     3%                NaN   
1      s2    Movie  07:19  Jorge Michel Grau   
2      s3    Movie  23:59       Gilbert Chan   
3      s4    Movie      9        Shane Acker   
4      s5    Movie     21     Robert Luketic   

                                                Cast        Country  \
0  João Miguel, Bianca Comparato, Michel Gomes, R...         Brazil   
1  Demián Bichir, Héctor Bonilla, Oscar Serrano, ...         Mexico   
2  Tedd Chan, Stella Chung, Henley Hii, Lawrence ...      Singapore   
3  Elijah Wood, John C. Reilly, Jennifer Connelly...  United States   
4  Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...  United States   

        Release_Date Rating   Duration  \
0    August 14, 2020  TV-MA  4 Seasons   
1  December 23, 2016  TV-MA     93 min   
2  December 20, 2018      R     78 min   
3  November 16, 2017  PG-13     80 min   
4    January 1, 2020  PG-13    123 min   

                               

In [4]:
print(data.isnull().sum())

Show_Id            0
Category           0
Title              0
Director        2388
Cast             718
Country          507
Release_Date      10
Rating             7
Duration           0
Type               0
Description        0
dtype: int64


In [5]:
data = data[["Title", "Description", "Category", "Type"]]
print(data.head())

   Title                                        Description Category  \
0     3%  In a future where the elite inhabit an island ...  TV Show   
1  07:19  After a devastating earthquake hits Mexico Cit...    Movie   
2  23:59  When an army recruit is found dead, his fellow...    Movie   
3      9  In a postapocalyptic world, rag-doll robots hi...    Movie   
4     21  A brilliant group of students become card-coun...    Movie   

                                                Type  
0  International TV Shows, TV Dramas, TV Sci-Fi &...  
1                       Dramas, International Movies  
2                Horror Movies, International Movies  
3  Action & Adventure, Independent Movies, Sci-Fi...  
4                                             Dramas  


In [6]:
data = data.dropna()

In [7]:
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Title"] = data["Title"].apply(clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
print(data.Title.sample(10))

7007                     witch part   subvers
5234    rob schneider asian momma mexican kid
3175                   jim norton mouth shame
2969                           search fellini
4087              mighti morphin alien ranger
7513                                    marin
7731                                      get
4572                     oddbod parti monster
4686                                   osmosi
2105                              famili busi
Name: Title, dtype: object


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
feature = data["Type"].tolist()
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(feature)
similarity = cosine_similarity(tfidf_matrix)

In [10]:
indices = pd.Series(data.index, 
                    index=data['Title']).drop_duplicates()

In [11]:
def netFlix_recommendation(title, similarity = similarity):
    index = indices[title]
    similarity_scores = list(enumerate(similarity[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[0:10]
    movieindices = [i[0] for i in similarity_scores]
    return data['Title'].iloc[movieindices]

print(netFlix_recommendation("krishna leela"))

249           special love
441             alway mayb
711              balu mahi
1371         christma view
2167    final found someon
3063           isa pa feel
3251                   way
3442               koshish
3453         krishna leela
3482       kyun ho gaya na
Name: Title, dtype: object
